In [1]:
from langchain_community.document_loaders import PyPDFLoader
pdf_path = "data/income_tax.pdf"
loader = PyPDFLoader(pdf_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
for i, page in enumerate(pages):
    if '55조(세율)' in page.page_content:
        print(i)

In [ ]:
pages[35].page_content
# 테이블이나 이미지는 파싱이 안된다 -> pyzerox 사용

In [ ]:
#!uv add -q py-zerox

In [2]:
from pyzerox import zerox
import os
import json
from dotenv import load_dotenv
load_dotenv()
import asyncio

In [4]:
#!uv add nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
## placeholder for additional model kwargs which might be required for some models
kwargs = {}

## system prompt to use for the vision model
custom_system_prompt = None

# to override
# custom_system_prompt = "For the below PDF page, do something..something..." ## example

###################### Example for OpenAI ######################
model = "gpt-4o-mini" ## openai model
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## GET CREDENTIALS
## RUN ##
# !gcloud auth application-default login - run this to add vertex credentials to your env
## OR ##

###################### For other providers refer: https://docs.litellm.ai/docs/providers ######################

# Define main async entrypoint
async def main():
    file_path = pdf_path ## local filepath and file URL supported

    ## process only some pages or all
    select_pages = None ## None for all, but could be int or list(int) page numbers (1 indexed)

    output_dir = "data/" ## directory to save the consolidated markdown file
    result = await zerox(
        file_path=file_path,
        model=model,
        output_dir=output_dir,
        custom_system_prompt=custom_system_prompt,
        select_pages=select_pages,
        concurrency=2,   # ✅ 무조건 1 또는 2로 제한
        **kwargs
)
    return result


# run the main function:
result = asyncio.run(main())

# print markdown result
print(result)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 100,
    separators = ['\n\n','\n']
)

In [14]:
!uv add markdown

Resolved 139 packages in 164ms                                       
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/105.15 KiB          
⠙ Preparing packages... (0/1)-------------- 16.00 KiB/105.15 KiB        
⠙ Preparing packages... (0/1)-------------- 32.00 KiB/105.15 KiB        
⠙ Preparing packages... (0/1)-------------- 48.00 KiB/105.15 KiB        
⠙ Preparing packages... (0/1)----------- 64.00 KiB/105.15 KiB        
⠙ Preparing packages... (0/1)---------- 80.00 KiB/105.15 KiB        
⠙ Preparing packages... (0/1)---------- 96.00 KiB/105.15 KiB        
Prepared 1 package in 38ms                                                   
Installed 1 package in 4ms                                  
 + markdown==3.10


In [15]:
#text splitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document


md_path = 'data/income_tax.md'
loader = UnstructuredMarkdownLoader(md_path)
doc_list = loader.load_and_split(text_splitter)

In [18]:
len(doc_list)

154